In [1]:

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 3.4MB/s 


In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls '/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/'

drive_csv_file.zip	 trip_csv_file.zip  weather_csv_file.zip
drive_vehcile_df_v1.csv  vehicle.csv


### Load Packages

In [0]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import os
from pathlib import Path
import datetime
from datetime import timedelta

In [0]:
drive_df = pd.read_csv('/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/drive_csv_file.zip')

In [6]:
drive_df.head(3)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
0,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:00,0.00,48.0,76.22,78.92,97.0,223.21,170.91,88.0,2066.48
1,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:01,44.52,55.0,75.44,83.57,93.0,211.91,167.83,75.0,2057.53
2,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:02,60.93,49.0,68.18,83.03,99.0,224.20,177.68,94.0,2046.12


In [7]:
drive_df.shape

(9217528, 12)

In [0]:
trip_df = pd.read_csv('/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/trip_csv_file.zip')

In [9]:
trip_df.head(3)

,vehicle_id,trip_id,datetime,lat,long,velocity
0,1000516,2e2a9936393a424abf4dea3943c83040,2017-01-01 07:00:00,31.4375,-85.437500,0.00
1,1000516,2e2a9936393a424abf4dea3943c83040,2017-01-01 07:00:01,31.4375,-85.437222,63.41
2,1000516,2e2a9936393a424abf4dea3943c83040,2017-01-01 07:00:02,31.4375,-85.436944,90.37


In [0]:
vehicle_df = pd.read_csv('/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/vehicle.csv')

In [11]:
vehicle_df.head(3)

,vehicle_id,year,make,Model,drivetrain,max_torque,max_horsepower,max_horsepower_rpm,max_torque_rpm,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation
0,1000500,2016,Honda,Civic,2,174,140,6500,1500,1.799,1059,47,16.50,20.0,4,1054.0,5
1,1000501,2016,Jeep,Compass,4,350,171,3750,1250,1.956,1059,60,17.10,21.2,3,NaN,4
2,1000502,2016,Hyundai,Creta,2,260,126,4000,1500,1.582,1059,55,19.67,24.1,4,1054.0,3


In [0]:
weather_df = pd.read_csv('/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/weather_csv_file.zip')

In [13]:
weather_df.head(3)

,x,y,date,time,lat,lon,temperature_data,temperature_unit,precipitation_data,precipitation_unit,wind_ew_data,wind_ew_unit,wind_ns_data,wind_ns_unit
0,127,39,2017-02-25,11Z,29.9375,-109.0625,278.38,k,0.0,kg/m^2,-1.07,m/s,-0.47,m/s
1,192,51,2017-01-02,18Z,31.4375,-100.9375,289.84,k,0.0,kg/m^2,5.74,m/s,-0.68,m/s
2,254,48,2017-02-23,19Z,31.0625,-93.1875,298.24,k,0.0,kg/m^2,-1.47,m/s,5.61,m/s


#### Prepare the Drive Data and Vehicle Data (Combined)

In [14]:
print(drive_df.shape)
print(vehicle_df.shape)

(9217528, 12)
(7, 17)


In [15]:
#identify the vehicleIds intersection

drive_df_vehicles = np.array(drive_df['vehicle_id'].unique().tolist())
drive_df_vehicles

print('The total number of vehicle_id from drive_df are :', drive_df_vehicles.size)

drive_df_vehicles = np.array(vehicle_df['vehicle_id'].unique().tolist())
drive_df_vehicles
print('The total number of  vehicle_ids from Vehicle dataframe are :', drive_df_vehicles.size)

## Get the common vehicles between train and test
common_vehicles = (np.intersect1d(drive_df_vehicles, drive_df_vehicles))
print('The total number of common vehicles are :',common_vehicles.size)

The total number of vehicle_id from drive_df are : 20
The total number of  vehicle_ids from Vehicle dataframe are : 7
The total number of common vehicles are : 7


In [0]:
drive_vehcile_df = pd.merge(drive_df, vehicle_df, on='vehicle_id',how='left')

In [0]:
# fill all values with NA as 0

drive_vehcile_df = drive_vehcile_df.fillna(0)

**Hints:**

Convert timezone to PST before any calculations
All vehicles from drive data should be in the final output even if you do not have specifications (Fill with 0 if specs are not given)
Active horsepower - Engine load / 255 * Max Torque * RPM / 5252
Horsepower utilization – Active horsepower / Max Horsepower
Torque Utilization - calculated as Engine load/ 255
RPM Utilization – RPM / Maximum horsepower rpm

In [18]:
drive_vehcile_df.tail(3)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm,year,make,Model,drivetrain,max_torque,max_horsepower,max_horsepower_rpm,max_torque_rpm,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation
9217525,1000519,4c4cf7c4219948f49d21a9394583c426,2017-01-03 03:21:33,74.27,85.56,81.22,79.87,98.0,205.84,100.74,98.26,1998.69,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9217526,1000519,4c4cf7c4219948f49d21a9394583c426,2017-01-03 03:21:34,20.00,96.99,92.37,80.73,88.0,204.10,92.88,102.06,2004.99,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9217527,1000519,4c4cf7c4219948f49d21a9394583c426,2017-01-03 03:21:35,0.00,93.33,84.64,77.46,90.0,221.13,99.28,99.52,1998.28,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
drive_vehcile_df.isnull().sum()

vehicle_id              0
trip_id                 0
datetime                0
velocity                0
accel_x                 0
accel_y                 0
accel_z                 0
engine_coolant_temp     0
eng_load                0
fuel_level              0
iat                     0
rpm                     0
year                    0
make                    0
Model                   0
drivetrain              0
max_torque              0
max_horsepower          0
max_horsepower_rpm      0
max_torque_rpm          0
engine_displacement     0
fuel_type               0
fuel_tank_capacity      0
fuel_economy_city       0
fuel_economy_highway    0
cylinders               0
forced_induction        0
device_generation       0
dtype: int64

In [0]:
# Prepare the requried columns
drive_vehcile_df['Active_horsepower'] = (drive_vehcile_df['eng_load'] * drive_vehcile_df['max_torque'] * drive_vehcile_df['rpm']) / (255*5252)
drive_vehcile_df['HorsePower_Utilization'] = drive_vehcile_df['Active_horsepower'] / drive_vehcile_df['max_horsepower']
drive_vehcile_df['Torque_Utilization'] = drive_vehcile_df['eng_load'] /255
drive_vehcile_df['RPM_Utilization'] = drive_vehcile_df['rpm'] / drive_vehcile_df['max_horsepower_rpm']

In [21]:
drive_vehcile_df.head(3)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm,year,make,Model,drivetrain,max_torque,max_horsepower,max_horsepower_rpm,max_torque_rpm,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation,Active_horsepower,HorsePower_Utilization,Torque_Utilization,RPM_Utilization
0,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:00,0.00,48.0,76.22,78.92,97.0,223.21,170.91,88.0,2066.48,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.875333,inf
1,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:01,44.52,55.0,75.44,83.57,93.0,211.91,167.83,75.0,2057.53,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.831020,inf
2,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:02,60.93,49.0,68.18,83.03,99.0,224.20,177.68,94.0,2046.12,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.879216,inf


In [0]:
# fill all values with NA as 0
drive_vehcile_df = drive_vehcile_df.replace([np.inf, -np.inf, np.nan], 0)

In [23]:
drive_vehcile_df.head(3)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm,year,make,Model,drivetrain,max_torque,max_horsepower,max_horsepower_rpm,max_torque_rpm,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation,Active_horsepower,HorsePower_Utilization,Torque_Utilization,RPM_Utilization
0,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:00,0.00,48.0,76.22,78.92,97.0,223.21,170.91,88.0,2066.48,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.875333,0.0
1,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:01,44.52,55.0,75.44,83.57,93.0,211.91,167.83,75.0,2057.53,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.831020,0.0
2,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 12:00:02,60.93,49.0,68.18,83.03,99.0,224.20,177.68,94.0,2046.12,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.879216,0.0


In [24]:
drive_vehcile_df.columns

Index(['vehicle_id', 'trip_id', 'datetime', 'velocity', 'accel_x', 'accel_y',
       'accel_z', 'engine_coolant_temp', 'eng_load', 'fuel_level', 'iat',
       'rpm', 'year', 'make', 'Model', 'drivetrain', 'max_torque',
       'max_horsepower', 'max_horsepower_rpm', 'max_torque_rpm',
       'engine_displacement', 'fuel_type', 'fuel_tank_capacity',
       'fuel_economy_city', 'fuel_economy_highway', 'cylinders',
       'forced_induction', 'device_generation', 'Active_horsepower',
       'HorsePower_Utilization', 'Torque_Utilization', 'RPM_Utilization'],
      dtype='object')

In [0]:
drive_vehcile_df_v1 = drive_vehcile_df[['vehicle_id', 'trip_id', 'datetime', 
                                        
       'HorsePower_Utilization', 'Torque_Utilization', 'RPM_Utilization']]

In [0]:
from datetime import datetime, time
def convert_datetime(x): 
  return datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S')  

In [0]:
from datetime import datetime
from pytz import timezone


def convert_date_timezone(x):  
  datetime_obj_naive = datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S')
  # Right way!
  datetime_obj_pacific = datetime_obj_naive.astimezone(timezone('US/Pacific'))
  return (datetime_obj_pacific.strftime("%Y-%m-%d %H:%M:%S"))

In [28]:
drive_vehcile_df_v1['datetime'] =  drive_vehcile_df_v1['datetime'].apply(lambda x :convert_date_timezone(x)) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
drive_vehcile_df_v1.tail(3)

,vehicle_id,trip_id,datetime,HorsePower_Utilization,Torque_Utilization,RPM_Utilization
9217525,1000519,4c4cf7c4219948f49d21a9394583c426,2017-01-02 19:21:33,0.0,0.807216,0.0
9217526,1000519,4c4cf7c4219948f49d21a9394583c426,2017-01-02 19:21:34,0.0,0.800392,0.0
9217527,1000519,4c4cf7c4219948f49d21a9394583c426,2017-01-02 19:21:35,0.0,0.867176,0.0


In [30]:
drive_vehcile_df_v1.dtypes

vehicle_id                  int64
trip_id                    object
datetime                   object
HorsePower_Utilization    float64
Torque_Utilization        float64
RPM_Utilization           float64
dtype: object

In [0]:
from datetime import datetime, timedelta
def reduce_hrs_to_pst(x):
  date_obj = datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
  date_obj = date_obj - timedelta(hours=8)
  return date_obj

In [32]:
drive_vehcile_df_v1["date"] = pd.to_datetime(drive_vehcile_df_v1["datetime"]).dt.date
# drive_vehcile_df_v1["time"] = pd.to_datetime(drive_vehcile_df_v1["datetime"]).dt.time

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
drive_vehcile_df_v1.dtypes

vehicle_id                  int64
trip_id                    object
datetime                   object
HorsePower_Utilization    float64
Torque_Utilization        float64
RPM_Utilization           float64
date                       object
dtype: object

In [0]:
# drive_vehcile_df_v1['datetime'] = pd.to_datetime(drive_vehcile_df_v1['datetime'])

In [0]:
# drive_vehcile_df_v1['date'] = pd.to_datetime(drive_vehcile_df_v1['date'])

In [0]:
# drive_vehcile_df_v1['date'] = pd.to_datetime(drive_vehcile_df_v1['date']) - pd.to_timedelta(7, unit='d')
# drive_vehcile_df_v1 = drive_vehcile_df_v1.groupby(['vehicle_id', pd.Grouper(key='date', freq='W-MON')]).sort_values('date')
       
       

In [0]:
# Python's program to get start and end of week 
from datetime import datetime, timedelta

def start_end_datetime(x):  
  date_obj = datetime.strptime(str(x), '%Y-%m-%d')
  return date_obj - timedelta(days=date_obj.weekday())  # Monday

In [37]:
drive_vehcile_df_v1['week_start_date'] = drive_vehcile_df_v1['date'].apply(lambda x:start_end_datetime(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
drive_vehcile_df_v1.head(4)

,vehicle_id,trip_id,datetime,HorsePower_Utilization,Torque_Utilization,RPM_Utilization,date,week_start_date
0,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 04:00:00,0.0,0.875333,0.0,2017-01-27,2017-01-23
1,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 04:00:01,0.0,0.831020,0.0,2017-01-27,2017-01-23
2,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 04:00:02,0.0,0.879216,0.0,2017-01-27,2017-01-23
3,1000517,e117220c8b15455f800a4b5b0c4c52a7,2017-01-27 04:00:03,0.0,0.869647,0.0,2017-01-27,2017-01-23


In [39]:
drive_vehcile_df_v1.isnull().sum()

vehicle_id                0
trip_id                   0
datetime                  0
HorsePower_Utilization    0
Torque_Utilization        0
RPM_Utilization           0
date                      0
week_start_date           0
dtype: int64

In [0]:
# Filter only few columns - which are required

drive_vehcile_df_Filterd = drive_vehcile_df_v1[['vehicle_id', 'week_start_date', 'HorsePower_Utilization',
                                                'Torque_Utilization',
                                                'RPM_Utilization', 'datetime' ]]
del drive_vehcile_df_v1

In [41]:
drive_vehcile_df_Filterd.shape

(9217528, 6)

In [42]:
drive_vehcile_df_Filterd.dtypes

vehicle_id                         int64
week_start_date           datetime64[ns]
HorsePower_Utilization           float64
Torque_Utilization               float64
RPM_Utilization                  float64
datetime                          object
dtype: object

In [0]:
drive_vehcile_df_Filterd['week_start_date'] = pd.to_datetime(drive_vehcile_df_Filterd['week_start_date'])
drive_vehcile_df_Filterd['datetime'] = pd.to_datetime(drive_vehcile_df_Filterd['datetime'])
# drive_vehcile_df_Filterd['date'] = pd.to_datetime(drive_vehcile_df_Filterd['date'])

In [0]:
drive_vehcile_df_Filterd = drive_vehcile_df_Filterd.sort_values(['vehicle_id', 'week_start_date', 'datetime'], ascending=[True, True, True])

In [0]:
# Transform the data into lists for each of the columns with an unique index column as trip_id
drive_vehcile_df_Filt_lst = drive_vehcile_df_Filterd.groupby(['vehicle_id', 'week_start_date']).agg(pd.Series.tolist).reset_index()

In [46]:
drive_vehcile_df_Filt_lst.head(3)

,vehicle_id,week_start_date,HorsePower_Utilization,Torque_Utilization,RPM_Utilization,datetime
0,1000500,2016-12-26,"[0.3936873357696467, 0.39103472221226654, 0.40...","[0.7705098039215685, 0.7678039215686274, 0.802...","[0.33217230769230766, 0.33109692307692307, 0.3...","[2017-01-01 10:00:00, 2017-01-01 10:00:01, 201..."
1,1000500,2017-01-02,"[0.39538272391149915, 0.4068294497974361, 0.41...","[0.8189411764705883, 0.8411372549019608, 0.862...","[0.31387384615384617, 0.3144384615384615, 0.31...","[2017-01-02 05:00:00, 2017-01-02 05:00:01, 201..."
2,1000500,2017-01-09,"[0.3906749531831011, 0.38482597518672357, 0.37...","[0.7716078431372548, 0.7564705882352941, 0.746...","[0.3291615384615385, 0.33072153846153846, 0.33...","[2017-01-09 00:00:00, 2017-01-09 00:00:01, 201..."


#### Other Fields Preparation

In [0]:
def convert_to_seconds(x):
  return x.days * 24 * 3600 + x.seconds  

In [0]:
def Calculate_Time(arr, timearr):
  global minimum_range
  global maximum_range
  n = 0
  nDiffSumofTimes = 0
  nTotalTime = 0
  datetimelist_updated = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in timearr]
  
  for i in range(0,len(datetimelist_updated)):
    if(arr[i] >= minimum_range and arr[i] < maximum_range and (i-1) >= 0):
#       diff_Times = datetimelist_updated[i] - datetimelist_updated[i-1]
      nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[i] - datetimelist_updated[i-1]).total_seconds())
#       nTotalTime = nTotalTime + int((datetimelist_updated[i] - datetimelist_updated[i-1]).total_seconds()) 
#       nTotalTime = nTotalTime + (diff_Times.days * 24 * 3600 + diff_Times.seconds) 
#       nDiffSumofTimes = nDiffSumofTimes + int((datetimelist_updated[i]).total_seconds() - (datetimelist_updated[i-1]).total_seconds())
  return nDiffSumofTimes

In [0]:
minimum_range = 0.6
maximum_range = 0.7
drive_vehcile_df_Filt_lst['ft_torque_util_60pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['Torque_Utilization'], row['datetime']), axis=1)

In [50]:
drive_vehcile_df_Filt_lst.head(3)

,vehicle_id,week_start_date,HorsePower_Utilization,Torque_Utilization,RPM_Utilization,datetime,ft_torque_util_60pct_s
0,1000500,2016-12-26,"[0.3936873357696467, 0.39103472221226654, 0.40...","[0.7705098039215685, 0.7678039215686274, 0.802...","[0.33217230769230766, 0.33109692307692307, 0.3...","[2017-01-01 10:00:00, 2017-01-01 10:00:01, 201...",0
1,1000500,2017-01-02,"[0.39538272391149915, 0.4068294497974361, 0.41...","[0.8189411764705883, 0.8411372549019608, 0.862...","[0.31387384615384617, 0.3144384615384615, 0.31...","[2017-01-02 05:00:00, 2017-01-02 05:00:01, 201...",163
2,1000500,2017-01-09,"[0.3906749531831011, 0.38482597518672357, 0.37...","[0.7716078431372548, 0.7564705882352941, 0.746...","[0.3291615384615385, 0.33072153846153846, 0.33...","[2017-01-09 00:00:00, 2017-01-09 00:00:01, 201...",37


In [51]:
drive_vehcile_df_Filt_lst['ft_torque_util_60pct_s'].min()

0

In [0]:
minimum_range = 0.7
maximum_range = 0.8
drive_vehcile_df_Filt_lst['ft_torque_util_70pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['Torque_Utilization'], row['datetime']), axis=1)

In [0]:
minimum_range = 0.8
maximum_range = 0.9
drive_vehcile_df_Filt_lst['ft_torque_util_80pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['Torque_Utilization'], row['datetime']), axis=1)

In [0]:
minimum_range = 0.9
maximum_range = 1.0
drive_vehcile_df_Filt_lst['ft_torque_util_90pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['Torque_Utilization'], row['datetime']), axis=1)

In [0]:
minimum_range = 0.5
maximum_range = 0.6
drive_vehcile_df_Filt_lst['ft_horsepower_util_50pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['HorsePower_Utilization'], row['datetime']), axis=1)

In [0]:
minimum_range = 0.6
maximum_range = 0.7
drive_vehcile_df_Filt_lst['ft_horsepower_util_60pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['HorsePower_Utilization'], row['datetime']), axis=1)

In [0]:
minimum_range = 0.7
maximum_range = 0.8
drive_vehcile_df_Filt_lst['ft_horsepower_util_70pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['HorsePower_Utilization'], row['datetime']), axis=1)

In [0]:
minimum_range = 0.8
maximum_range = 0.9
drive_vehcile_df_Filt_lst['ft_horsepower_util_80pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['HorsePower_Utilization'], row['datetime']), axis=1)

In [0]:
minimum_range = 0.5
maximum_range = 0.6
drive_vehcile_df_Filt_lst['ft_rpm_util_50pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['RPM_Utilization'], row['datetime']), axis=1)

In [0]:
minimum_range = 0.6
maximum_range = 0.7
drive_vehcile_df_Filt_lst['ft_rpm_util_60pct_s']  = drive_vehcile_df_Filt_lst.apply(lambda row: Calculate_Time(row['RPM_Utilization'], row['datetime']), axis=1)

In [61]:
drive_vehcile_df_Filt_lst.head(3)

,vehicle_id,week_start_date,HorsePower_Utilization,Torque_Utilization,RPM_Utilization,datetime,ft_torque_util_60pct_s,ft_torque_util_70pct_s,ft_torque_util_80pct_s,ft_torque_util_90pct_s,ft_horsepower_util_50pct_s,ft_horsepower_util_60pct_s,ft_horsepower_util_70pct_s,ft_horsepower_util_80pct_s,ft_rpm_util_50pct_s,ft_rpm_util_60pct_s
0,1000500,2016-12-26,"[0.3936873357696467, 0.39103472221226654, 0.40...","[0.7705098039215685, 0.7678039215686274, 0.802...","[0.33217230769230766, 0.33109692307692307, 0.3...","[2017-01-01 10:00:00, 2017-01-01 10:00:01, 201...",0,6532,28102,6,0,0,0,0,0,0
1,1000500,2017-01-02,"[0.39538272391149915, 0.4068294497974361, 0.41...","[0.8189411764705883, 0.8411372549019608, 0.862...","[0.31387384615384617, 0.3144384615384615, 0.31...","[2017-01-02 05:00:00, 2017-01-02 05:00:01, 201...",163,415813,158965,119,0,0,0,0,0,0
2,1000500,2017-01-09,"[0.3906749531831011, 0.38482597518672357, 0.37...","[0.7716078431372548, 0.7564705882352941, 0.746...","[0.3291615384615385, 0.33072153846153846, 0.33...","[2017-01-09 00:00:00, 2017-01-09 00:00:01, 201...",37,62206,492717,0,0,0,0,0,0,0


In [62]:
drive_vehcile_df_Filt_lst.columns

Index(['vehicle_id', 'week_start_date', 'HorsePower_Utilization',
       'Torque_Utilization', 'RPM_Utilization', 'datetime',
       'ft_torque_util_60pct_s', 'ft_torque_util_70pct_s',
       'ft_torque_util_80pct_s', 'ft_torque_util_90pct_s',
       'ft_horsepower_util_50pct_s', 'ft_horsepower_util_60pct_s',
       'ft_horsepower_util_70pct_s', 'ft_horsepower_util_80pct_s',
       'ft_rpm_util_50pct_s', 'ft_rpm_util_60pct_s'],
      dtype='object')

In [0]:
engine_features = drive_vehcile_df_Filt_lst[['vehicle_id', 'week_start_date',
                                             'ft_torque_util_60pct_s', 'ft_torque_util_70pct_s',
                                          'ft_torque_util_80pct_s', 'ft_torque_util_90pct_s',
                                       'ft_horsepower_util_50pct_s', 'ft_horsepower_util_60pct_s',
                                      'ft_horsepower_util_70pct_s', 'ft_horsepower_util_80pct_s',
                                        'ft_rpm_util_50pct_s', 'ft_rpm_util_60pct_s' ]]

In [0]:
engine_features = engine_features.sort_values(['vehicle_id', 'week_start_date'], ascending=[True, True])

In [0]:
engine_features.to_csv('engine_features.csv',index=False)